In [1]:
!pip install tensorflow gym==0.19 keras keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import gym 
import random 

In [3]:
env=gym.make('CartPole-v0')
states=env.observation_space.shape[0]
actions=env.action_space.n

In [4]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender
!pip install imageio==2.4.1
!apt-get install python-opengl -y

!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

!apt install xvfb -y

!pip install pyvirtualdisplay

!pip install piglet

!pip install pyglet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [5]:
import random
from gym.wrappers import Monitor
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

# Google Colab needs to render the environment to a virtual display
# we will record this as a video and play it after the training has finished
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[-1]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [6]:
!pip uninstall pyglet
!pip install pyglet==1.5.27

Found existing installation: pyglet 1.5.27
Uninstalling pyglet-1.5.27:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/pyglet-1.5.27.dist-info/*
    /usr/local/lib/python3.8/dist-packages/pyglet/*
Proceed (y/n)? y
  Successfully uninstalled pyglet-1.5.27
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyglet-1.5.27-py3-none-any.whl (1.1 MB)


In [8]:
from colabgymrender.recorder import Recorder

directory='./video'


for episode in range(5):
  
  state = env.reset() 
  
  done=False 
  score =0 
  #env=Recorder(env,directory)
  while not done:
    action=random.choice([0,1])
    n_state,reward,done,info = env.step(action)
    score+=reward 
  #env.play()
  print("Episodes:",episode," Score:",score)
#display.clear_output(wait=True)
env.close()

Episodes: 0  Score: 30.0
Episodes: 1  Score: 33.0
Episodes: 2  Score: 16.0
Episodes: 3  Score: 34.0
Episodes: 4  Score: 32.0


In [9]:
import numpy as np 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [16]:
def build_model(states,action):
  model=Sequential()
  model.add(Flatten(input_shape=(1,states)))
  model.add(Dense(24,activation='relu'))
  model.add(Dense(24,activation='relu'))
  model.add(Dense(actions,activation='linear'))
  return model

In [17]:
model=build_model(states,actions)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [19]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory 
from rl.policy import BoltzmannQPolicy

In [20]:
def build_agent(model,actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000,window_length=1)
  dqn = DQNAgent(model=model,memory=memory,policy=policy,nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
  return dqn

In [21]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-4),metrics=['mae'])
dqn.fit(env,nb_steps=10000,visualize=False,verbose=2)

  updates=self.state_updates,



Training for 10000 steps ...


  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')

  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')



   15/10000: episode: 1, duration: 1.080s, episode steps:  15, steps per second:  14, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.733 [0.000, 1.000],  loss: 0.586483, mae: 0.633133, mean_q: 0.054173
   37/10000: episode: 2, duration: 0.214s, episode steps:  22, steps per second: 103, episode reward: 22.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  loss: 0.600445, mae: 0.630490, mean_q: 0.067853
   54/10000: episode: 3, duration: 0.173s, episode steps:  17, steps per second:  98, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.235 [0.000, 1.000],  loss: 0.558230, mae: 0.581625, mean_q: 0.067712
   66/10000: episode: 4, duration: 0.111s, episode steps:  12, steps per second: 108, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.917 [0.000, 1.000],  loss: 0.516043, mae: 0.565932, mean_q: 0.094708
   79/10000: episode: 5, duration: 0.128s, episode steps:  13, steps per

In [22]:
score = dqn.test(env,nb_episodes=100,visualize=False)
np.mean(score.history['episode_reward'])

Testing for 100 episodes ...
Episode 1: reward: 65.000, steps: 65
Episode 2: reward: 62.000, steps: 62
Episode 3: reward: 65.000, steps: 65
Episode 4: reward: 53.000, steps: 53
Episode 5: reward: 52.000, steps: 52
Episode 6: reward: 55.000, steps: 55
Episode 7: reward: 79.000, steps: 79
Episode 8: reward: 63.000, steps: 63
Episode 9: reward: 106.000, steps: 106
Episode 10: reward: 63.000, steps: 63
Episode 11: reward: 51.000, steps: 51
Episode 12: reward: 52.000, steps: 52
Episode 13: reward: 53.000, steps: 53
Episode 14: reward: 58.000, steps: 58
Episode 15: reward: 75.000, steps: 75
Episode 16: reward: 49.000, steps: 49
Episode 17: reward: 71.000, steps: 71
Episode 18: reward: 64.000, steps: 64
Episode 19: reward: 102.000, steps: 102
Episode 20: reward: 58.000, steps: 58
Episode 21: reward: 68.000, steps: 68
Episode 22: reward: 77.000, steps: 77
Episode 23: reward: 57.000, steps: 57
Episode 24: reward: 58.000, steps: 58
Episode 25: reward: 82.000, steps: 82
Episode 26: reward: 96.000

78.44